In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

# Model building. "Timeseries" case study: Forecasting oil spot price

##Goals / Topics
 - Trend
 - Seasonal effects / Fourier transform
 - Lagged inputs, autocorrelation
 - Testing with timeseries data
 - Autoregressive models, VAR

##The problem: Timeseries forecasting

Try to predict the daily average _spot price for crude oil_ three months (:=60 market days) out.  For a change, we will also try to predict _hourly temperature data_ (for Pittsburgh) 24 hours out.


**The type of learner**: This is a _supervised regression_ problem.

**The training dataset**: We'll get historical oil spot price data from Quandl.  We'll get historical hourly temperate data (the NWS Pittsburgh Climate Data) from NOAA.  Time allowing, we could try throwing in some non-financial signals, namely "political event" data from [The Global Database of Events, Language, and Tone (GDELT)](http://gdeltproject.org/)

**The test dataset**: We will generate a holdout data set from the training set.

In [ ]:
import Quandl
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.metrics
import statsmodels.api as sm

In [ ]:
# To really use the Quandl API, you should get an authtoken.  Limited usage doesn't require it.

authtoken = None
# authtoken = "your token here"


def getQuandl(what):
    """ 
    Wrapper around Quandl requests, using authtoken only if available
    """
    if authtoken:
        return Quandl.get(what, authtoken=authtoken)
    else:
        return Quandl.get(what)

In [ ]:
oil = getQuandl("DOE/RWTC")

In [ ]:
oil.plot()

In [ ]:
## Create a column with "true" (future) values
PERIOD_MONTH = 20
PREDICTION_LAG = 3 * PERIOD_MONTH

CUT_YEAR = 2008
oil['Actual'] = oil['Value'].shift(-PREDICTION_LAG)

## First Step: Take out any 'inflationary' effects.
Suppose, instead, we'd been asked to predict the value **one year** or **five years** out.  The simplest model that we might try is just exponential growth
   $$ P(t) = \exp\left( c_0 + c_1 t \right) $$

To train it we just use a linear regression on $\log P(t)$ to build this model.  If we want to do better, we can content ourselves with modeling its error term.

In [ ]:
oil['Julian'] = oil.index.to_julian_date()
oil = sm.add_constant(oil) # Add a constant field for the linear regression

In [ ]:
train = oil[oil.index.year < CUT_YEAR].dropna(how='any')
train.head()

In [ ]:
# Some justification is required for not doing in-/out-sample here...
#train = oil[ oil.index.year < CUT_YEAR ].dropna(how='any')
train = oil
exponential_model = sklearn.linear_model.Ridge().fit( 
    X=train[['Julian', 'const']], 
    y=np.log(train['Value'])
)

exp_model_df = oil
exp_model_df['Exponential_Model'] = np.exp(exponential_model.predict(oil[['Julian', 'const']]))
exp_model_df['Log_Error_Exponential'] = np.log(oil['Value'] / oil['Exponential_Model'])

In [ ]:
exp_model_df[['Value', 'Exponential_Model']].plot()

##Step 2: Seasonal effects
We might guess that the price of oil goes up in the winter and down in the summer.  Of course, oil can be stored and in an efficient futures market this would be priced in -- so we might _not_ expect to see it.  Let's do two analyses to try to check for it:
 - Visual inspection (with a weighted average, for smoothing)
 - A Fast Fourier Transform
 
(Later on, we will also look at the _autocorrelation_ which is another technique for this.)

## Weighted average (with smoothing)
There are many types of [moving average](http://en.wikipedia.org/wiki/Moving_average).  We will be computing the exponentially weighted moving average:

$$E_t = \alpha X_t + (1-\alpha)E_{t-1}$$

Since we can see that something crazy happened in 2008 (financial meltdown) and something else crazy happened in 1992 (Iraq War), we'll restrict to between those two dates.

In [ ]:
log_error = oil['Log_Error_Exponential'][(oil.index.year > 1992) & (oil.index.year < 2008)]
pd.DataFrame({
    'log_error': log_error,
    'ewma': pd.ewma(log_error, span=100)
}).plot()

# FFT

We can also take a **Fourier transform** (more precisely, an FFT).

The Fourier transform can be thought of as a representation of all the frequency components of your data. In some sense it is a histogram with each “frequency bin” corresponding to how often a particular frequency occurs in your signal.

In [ ]:
from scipy import fftpack

fft = fftpack.fft(oil['Log_Error_Exponential'][(oil.index.year > 1992) & (oil.index.year < 2010)])
plt.plot(np.abs(fft))
plt.ylim([0, 700])
plt.xlim([0, 400])

**So what?**  There seems to be pretty much no visible season effect...

**Exercise:** Suppose we had use something that _is_ clearly seasonal (e.g., the temperature data in the next cell).  Carry out the above analysis (and the lagged auto-correlation one below) with the temperature data below, and confirm that it is reasonable.

In [ ]:
!head projects/timeseries-project/data/raw/temperatures.csv

In [ ]:
temps_df = pd.read_csv("projects/timeseries-project/data/raw/temperatures.csv", 
                       index_col=0,
                       names=["Temperature"],
                       parse_dates=True,
                       date_parser=lambda u: pd.datetime.strptime(u, "%Y-%m-%d %H:%M:%S"))
temps_df[:24 * 7].plot()

In [ ]:
## Your code here ##

## Cross validation: In sample and out of sample data sets.

In the above, when we built our simple model, we used all of history for training (and if we did any testing, we again used all of history for that).  When building a predictor, we should be doing some cross validation to make sure that we do not overfit.  

How do we do cross validation for time series data?  Here are some things we generally cannot do:
 - We cannot just pick data points at random, because there might be lagged indicators / seasonal effects / etc. that force us to work with contiguous blocks of time.  
 - We cannot blindly chop by e.g., month or year without some thought: There could be seasonal effects so that Decembers are always different.  There could be systemic "regime changes" that mean that cutting at a given date is inappropriate, or known and time-limited effects that last a year (or fraction thereof).   For instance, the years 1991 and 2008 in this data set.
 - We cannot have our testing set occur before our training set.
 
One common technique is to use a rolling window, sometimes called [forward chaining](http://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection).

- fold 1 : training [1], test [2]
- fold 2 : training [1 2], test [3]
- fold 3 : training [1 2 3], test [4]
- fold 4 : training [1 2 3 4], test [5]
- fold 5 : training [1 2 3 4 5], test [6]

For consistency, you often keep the training window size fixed.
 
We will pick the following training and testing sets:
 - **Train**: years <2008
 - **Test**: years 2008-present
 
We've picked this to be purposefully a little perverse: it includes the (crazy) price swings of 2008 in the testing set.
 
To start with, we will apply these techniques to a very simple first model:
  - A "benchmark" model: we will just use the current value.

In [ ]:
### Cross validation -- benchmark model

#Train/Test
train = oil[(oil.index.year < CUT_YEAR)].copy()
test = oil[(oil.index.year >= CUT_YEAR)].copy()

# Reporting function
def summarize_errors(test_me):
    error_pct = ((test_me['Actual'] - test_me['Model'])/test_me['Actual'])

    print error_pct.describe()
    error_pct.plot()
    plt.show()

    error_pct.hist(bins=100, normed=True)
    x = np.arange(-1, 0.5, 0.001)
    n_pdf = sp.stats.norm(loc=error_pct.mean(), scale=error_pct.std()).pdf
    plt.plot(x, n_pdf(x), linewidth=3, color='red')
    plt.show()

    print sklearn.metrics.mean_squared_error(test_me['Actual'], test_me['Model'])

test['Benchmark_Model'] = oil['Value']
test_me = test[['Actual', 'Benchmark_Model']].dropna(how='any') \
                                             .rename(columns={"Benchmark_Model": "Model"})
summarize_errors(test_me)

##Step 3: Lagged auto-correlation
For many time series, the best prediction for $t_{i+1}$ is $t_{i}$ or $t_{i-1}$.  We used something similar as our "benchmark" model above.

One proxy for measuring this is the __auto-correlation__: that is, the correlation between the sequences $t_{i+1}$ and $t_{i}$ (or more generally $t_i$ and $t_{i-\ell}$ for some lag $\ell$).

In [ ]:
from pandas.tools.plotting import autocorrelation_plot

autocorrelation_plot(oil.Value)

#### So what?
The autocorrelation starts off high, using the last few day's values is likely to be useful.  There are no extra "bumps" or peaks, consistent with our observation that there were no seasonal effects.  

Let's build a second model -- which we will call the "simple" model -- which is also auto-regressive but now takes into account our exponential model from above.  Namely: we will start with the exponential model from before, and then try to estimate __its error__ using an auto-regressive linear model.

In [ ]:
### Cross validation -- simple model

#Train/Test
train = oil[oil.index.year < CUT_YEAR]
test = oil[oil.index.year >= CUT_YEAR]

# Reporting function
def summarize_errors(test_me):
    error_pct = (test_me['Actual'] - test_me['Model']) / test_me['Actual']

    print error_pct.describe()
    error_pct.plot()
    plt.show()

    error_pct.hist(bins=100, normed=True)
    x = np.arange(-1, 1, 0.001)
    plt.plot(x, sp.stats.norm(loc=error_pct.mean(), scale=error_pct.std()).pdf(x), linewidth=3, color='red')
    plt.show()

    print sklearn.metrics.mean_squared_error( test_me['Actual'], test_me['Model'] )

# Train the regression
def frame_to_feats(frame):
    feats = pd.DataFrame()
    
    feats['LEE'] = frame['Log_Error_Exponential']
    feats['LEE_1'] = frame['Log_Error_Exponential'].shift(1)
    feats['dLEE_avg'] = pd.rolling_mean(frame['Value'].diff(), window=3*PERIOD_MONTH)
    feats['vol_avg'] = pd.ewmvar(frame['Value'], span=3*PERIOD_MONTH)
    
    feats['Actual_LEE'] = frame['Log_Error_Exponential'].shift(-PREDICTION_LAG)
    return sm.add_constant(feats)

feats = frame_to_feats(train).dropna(how='any')
regress = sklearn.linear_model.LinearRegression().fit( 
        X=feats.drop('Actual_LEE', axis=1).values, 
        y=feats['Actual_LEE'].values)

# Predict

feats = frame_to_feats(test).dropna(how='any')
feats['Predicted_LEE'] = regress.predict(X=feats.drop('Actual_LEE', axis=1))

test = feats.join(test, rsuffix='_r').dropna(how='any')
test['Simple_Model'] = np.exp(test['Predicted_LEE']) * test['Exponential_Model']

# Report
test_me = test[['Actual', 'Simple_Model']].dropna(how='any') \
                                          .rename(columns={'Simple_Model': 'Model'})
summarize_errors(test_me)

##Step 4: Adding external indicators (e.g., volatility, gdelt)

Once we have a  "simplest"  model as above, we can get to the interesting part:
At this point we like to find signal in additional data source that accounts for some of the error; to try to conceptually explain sources of error or skews in the distribution of error; etc.  Here are examples of other data sources we might try:

  - Other financial indicators (e.g., interest rates, volatilities, related commodities)
  - Non-financial indicators (e.g., weather, indicators for weather patterns / wars, geopolitical data like gdelt).
  
We'll show the example of trying to use equities volatility data (in the form of the VIX index) -- this will not help.

In [ ]:
ng_fut = getQuandl("CHRIS/CME_NG1")
vix = getQuandl("YAHOO/INDEX_VIX")

oil['vix'] = vix['Adjusted Close']
oil['ng_fut'] = ng_fut['Settle']

In [ ]:
print oil['Log_Error_Exponential'].corr(oil['vix'])  # Our error term does correlate negatively with vix...
print oil['Log_Error_Exponential'].corr(oil['ng_fut'])

oil['vix'].plot()
plt.show()

oil['ng_fut'].plot()
plt.show()

In [ ]:
### Cross validation -- complex model -- notice that we have overfit!

#Train/Test
train = oil[oil.index.year < CUT_YEAR]
test = oil[oil.index.year >= CUT_YEAR]

# Reporting function
def summarize_errors(test_me):
    error_pct = (test_me['Actual'] - test_me['Model']) / test_me['Actual']

    print error_pct.describe()
    error_pct.plot()
    plt.show()

    error_pct.hist(bins=100, normed=True)
    x = np.arange(-0.5, 0.5, 0.001)
    n_pdf = sp.stats.norm(loc=error_pct.mean(), scale=error_pct.std()).pdf
    plt.plot(x, n_pdf(x), linewidth=3, color='red')
    plt.show()

    print sklearn.metrics.mean_squared_error( test_me['Actual'], test_me['Model'] )

# Train the regression
def frame_to_feats(frame):
    feats = pd.DataFrame()
    
    feats['LEE'] = frame['Log_Error_Exponential']
    feats['dLEE_avg'] = pd.rolling_mean(frame['Value'].diff(), window=3*PERIOD_MONTH)
    feats['vol_avg'] = pd.ewmvar(frame['Value'], span=3*PERIOD_MONTH)
    
    feats['ng_fut'] = frame['ng_fut']
    
    feats['Actual_LEE'] = frame['Log_Error_Exponential'].shift(-PREDICTION_LAG)
    return sm.add_constant(feats)
    

feats = frame_to_feats(train).dropna(how='any')
regress = sklearn.linear_model.LinearRegression().fit( 
        X=feats.drop('Actual_LEE', axis=1), 
        y=feats['Actual_LEE'])

# Predict

feats = frame_to_feats(test).dropna(how='any')
feats['Predicted_LEE'] = regress.predict(feats.drop('Actual_LEE', axis=1))

test = feats.join(test, rsuffix='_r').dropna(how='any')
test['Complex_Model'] = np.exp (test['Predicted_LEE']) * test['Exponential_Model']

# Report
test_me = test[['Actual', 'Complex_Model']].dropna(how='any') \
                                           .rename(columns = {'Complex_Model': 'Model'})
summarize_errors(test_me)


### Open-ended brainstorming / exercises

1. What happens to the results above if we change our "cut point" to say 2010?  What's the moral of this story...

2. Play around with the previous "Complex" model, and see if you can improve it.  What happens, for instance, if you get rid of the 'vix' signal.  Why do you think this might be the case?

3. What are some other "simplest" models we could have tried? e .g., linear regression just on 'Value' rather than going through this log stuff.  Try some of them -- how do they perform?

4. Carry out the whole analysis process for the Pittsburgh temperate data (see below for some steps.. if you do use that, mark it up to explain what's happening).

##Fancier topic: Stochastic auto-regressive models

Our time series has, very cleary, time-varying volatility.  To accurately model these effects, one often uses stochastic models.  To start you Googling, the basic auto-regressive examples are **ARCH/GARCH**.  

Let us say just a little about these, leaving an example as an exercise to the reader.  In this type of model, the next time tick's value is drawn from a _distribution_ whose mean **and** standard deviation are modelled over time (and can, in general, be auto-regressive):

$$ t_{i+1} = M(\text{..factors..}) + \sigma(\text{..factors..}) \epsilon_t $$

where 
  - $M$ is some model for the mean (e.g., a linear model depending on some number of time lags of $t_{i}$ and moving averages in GARCH models);
  - $\sigma$ is some model for the standard deviation (as above in GARCH);
  - and, $\epsilon_t$ is a draw from a distribution having (conditional on the factors..) mean equal to zero, and standard deviation equal to one.  (In ARCH, this is a normal distribution.)
  
Stochastic models allow us to generate a range of future paths, for instance for modelling "value at risk."

##ARMA models: Combining auto-regressive and moving average

If a time series can be made stationary by differencing or by the methods above, it is common to model them using some combination of autoregressive terms (weighted average over some recent values) and moving average terms (weighted average over some recent errors) of different orders - in Python you can find this functionality in the statsmodels library. The number of terms can be determined through various methods and rules of thumb. [Read more.](http://people.duke.edu/~rnau/411arim.htm)

## A more positive example: Temperatures

In [ ]:
temps_df = pd.read_csv("projects/timeseries-project/data/raw/temperatures.csv", 
                       index_col=0,
                       names=["Temperature"],
                       parse_dates=True,
                       date_parser=lambda u: pd.datetime.strptime(u, "%Y-%m-%d %H:%M:%S"))

In [ ]:
ts = temps_df['Temperature'].asfreq('60Min', method='ffill')
temps_df['Temperature'] = ts

In [ ]:
# 2-month exponential moving average
pd.ewma(ts, span=2*30).plot()

In [ ]:
from scipy import fftpack
fft = fftpack.fft(ts)

In [ ]:
plt.plot(np.abs(fft))
plt.title("FFT of temperature data (zoomed out)")
plt.ylim([0,1000000])
plt.xlim([0,10000])

In [ ]:
plt.plot(np.abs(fft))
plt.title("FFT of temperature data (zoomed in)")
plt.ylim([0,1000000])
plt.xlim([0,24*2])

In [ ]:
plt.plot(np.array([ts.corr(ts.tshift(i)) for i in xrange(1, 24*5)]))
plt.title("Lagged autocorrelation (over five days)")
plt.ylabel("Correlation")
plt.xlabel("Lag (hours)")

In [ ]:
plt.plot(np.array([ts.corr(ts.tshift(i)) for i in xrange(1, 8766, 24)]))
plt.title("Lagged autocorrelation (over a year)")
plt.ylabel("Correlation")
plt.xlabel("Lag (days)")


We can always decompose
$$F(t) = k \cos(\omega (t - t_0)) = k \left[\alpha \cos(\omega t) + \beta \sin(\omega t) \right]$$
where $\alpha^2 + \beta^2 = 1$

In [ ]:
temps_df['Julian'] = temps_df.index.to_julian_date()
temps_df['const'] = 1
temps_df['sin(year)'] = np.sin(temps_df['Julian'] / 365.25 * 2 * np.pi)
temps_df['cos(year)'] = np.cos(temps_df['Julian'] / 365.25 * 2 * np.pi)
temps_df['sin(6mo)'] = np.sin(temps_df['Julian'] / (365.25 / 2) * 2 * np.pi)
temps_df['cos(6mo)'] = np.cos(temps_df['Julian'] / (365.25 / 2) * 2 * np.pi)
temps_df['sin(day)'] = np.sin(temps_df.index.hour / 24.0 * 2* np.pi)
temps_df['cos(day)'] = np.cos(temps_df.index.hour / 24.0 * 2* np.pi)
temps_df['Day_Average'] = pd.ewma(temps_df['Temperature'], span=24)

temps_df['Goal'] = temps_df['Temperature'].shift(-24)

In [ ]:
cut_year = 2012

train = temps_df[ temps_df.index.year < cut_year ].dropna(how='any')
test  = temps_df[ temps_df.index.year >= cut_year].dropna(how='any')

regress = sklearn.linear_model.LinearRegression().fit( 
        X=train[['Temperature']], 
        y=train['Goal'])

test['Predicted_Value'] = regress.predict(X=test[['Temperature']])

(test['Goal'] - test['Predicted_Value']).plot()
print sklearn.metrics.mean_squared_error(test['Goal'], test['Predicted_Value'])

In [ ]:
cut_year = 2012

train = temps_df[temps_df.index.year < cut_year].dropna(how='any')
test  = temps_df[temps_df.index.year >= cut_year].dropna(how='any')

regress = sklearn.linear_model.LinearRegression().fit( 
        X=train[['Temperature', 'Day_Average', 'sin(year)', 'cos(year)', 'sin(6mo)', 'cos(6mo)', 'sin(day)', 'cos(day)']], 
        y=train['Goal'])

test['Predicted_Value'] = regress.predict(X=test[['Temperature', 'Day_Average', 'sin(year)', 'cos(year)', 'sin(6mo)', 'cos(6mo)', 'sin(day)', 'cos(day)']] )

(test['Goal'] - test['Predicted_Value']).plot()
print sklearn.metrics.mean_squared_error(test['Goal'], test['Predicted_Value'])

In [ ]:
cut_year = 2012

train = temps_df[ temps_df.index.year < cut_year ].dropna(how='any')
test  = temps_df[ temps_df.index.year >= cut_year].dropna(how='any')

regress = sklearn.linear_model.LinearRegression().fit( 
        X=train[['Temperature', 'Day_Average', 'sin(year)', 'cos(year)', 'sin(day)', 'cos(day)']], 
        y=train['Goal'])

test['Predicted_Value'] = regress.predict(X=test[['Temperature', 'Day_Average', 'sin(year)', 'cos(year)', 'sin(day)', 'cos(day)']])

test[['Goal', 'Predicted_Value']].plot()
print sklearn.metrics.mean_squared_error(test['Goal'], test['Predicted_Value'])

plt.show()

#### Exit Tickets
1. Describe how you would cross-validate a time series model.
1. Describe the difference between autoregressive and moving average terms in an ARMA model.
1. Explain an FFT to a layman.

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*